## Experiment description

The goal of this experiment is to train a TM classifier trained on 40% of the data available in Wind Farm C, and then evaluate the model on the remaining 60% of the data. The model will be evaluated on the accuracy score.

In [5]:
# Import all required libraries

import numpy as np
import pickle

from tmu.models.classification.vanilla_classifier import TMClassifier

In [6]:
train_datasets = np.array([68, 22, 72, 73, 0])
test_datasets = np.array([26])

In [7]:
## Helper functions for saving the model and accuracy

# Helper function to save the model
def save_model(model, filename):
    with open(filename, "wb") as f:
        pickle.dump(model, f)


# Helper function to save the accuracy
def save_accuracy(epoch, accuracy, tp, tn, fp, fn):
    with open("accuracy.txt", "a") as f:
        f.write(f"{epoch},{accuracy},{tp},{tn},{fp},{fn}\n")


# Helper function to load dataset
def load_dataset(farm, event_id):
    X = np.loadtxt(f"./data_train/X_{farm}_{event_id}.txt", dtype=np.uint32)
    X = np.array(X).astype(np.uint32)
    return X


def load_dataset_labels(farm, event_id):
    y = np.loadtxt(f"./data_train/y_{farm}_{event_id}.txt", dtype=np.uint32)
    y = np.array(y).astype(np.uint32)
    return y


def load_test_dataset(farm, event_id):
    X = np.loadtxt(f"./data_test/X_{farm}_{event_id}.txt", dtype=np.uint32)
    X = np.array(X).astype(np.uint32)
    return X


def load_test_dataset_labels(farm, event_id):
    y = np.loadtxt(f"./data_test/y_{farm}_{event_id}.txt", dtype=np.uint32)
    y = np.array(y).astype(np.uint32)
    return y


def get_train_dataset():
    # Load all train datasets and concat them
    dataset = np.concatenate([load_dataset("A", dataset) for dataset in train_datasets])
    labels = np.concatenate([load_dataset_labels("A", dataset) for dataset in train_datasets])
    return dataset, labels


def get_test_dataset():
    # Load all train datasets and concat them
    dataset = np.concatenate([load_test_dataset("A", dataset) for dataset in test_datasets])
    labels = np.concatenate([load_test_dataset_labels("A", dataset) for dataset in test_datasets])
    return dataset, labels

In [8]:
epochs = 100
platform = "CPU"

tm = TMClassifier(
    number_of_clauses=6770,
    T=4838,
    s=1.0614727101769625,
    max_included_literals=1002,
    weighted_clauses=True,
    type_iii_feedback=False,
    platform=platform,
    batch_size=250,
)

best_accuracy = 0

print(f"Running {TMClassifier} for {epochs} epochs")

(X_train, y_train) = get_train_dataset()
(X_test, y_test) = get_test_dataset()

for epoch in range(epochs):

    tm.fit(X_train, y_train)

    pred = tm.predict(X_test)

    pred_0s = np.count_nonzero(pred == 0)
    pred_1s = np.count_nonzero(pred == 1)

    print(f"Predicted 0s: {pred_0s}, Predicted 1s: {pred_1s}")

    accuracy = np.sum(pred == y_test) / len(y_test)

    # the number of false positives based on g and p
    fp = np.sum((pred == 1) & (y_test == 0))

    # the number of true negatives based on g and p
    tn = np.sum((pred == 0) & (y_test == 0))

    # the number of true positives based on g and p
    tp = np.sum((pred == 1) & (y_test == 1))

    # the number of false negatives based on g and p
    fn = np.sum((pred == 0) & (y_test == 1))

    print(f"Accuracy = {accuracy:.5f}, TP = {tp}, TN = {tn}, FP = {fp}, FN = {fn}")

    save_accuracy(epoch, accuracy, tp, tn, fp, fn)

    # Print every 20 epochs
    #if (epoch + 1) % 20 == 0:
    print(f"Epoch: {epoch + 1}, Accuracy: {accuracy:.5f}")

    if accuracy > best_accuracy:
        best_accuracy = accuracy

        print("Saving model")
        save_model(tm, "best.pkl")


Running <class 'tmu.models.classification.vanilla_classifier.TMClassifier'> for 100 epochs
Predicted 0s: 760, Predicted 1s: 681
Accuracy = 0.49063, TP = 478, TN = 229, FP = 203, FN = 531
Epoch: 1, Accuracy: 0.49063
Saving model
Predicted 0s: 734, Predicted 1s: 707
Accuracy = 0.54060, TP = 527, TN = 252, FP = 180, FN = 482
Epoch: 2, Accuracy: 0.54060
Saving model
Predicted 0s: 498, Predicted 1s: 943
Accuracy = 0.62110, TP = 703, TN = 192, FP = 240, FN = 306
Epoch: 3, Accuracy: 0.62110
Saving model
Predicted 0s: 552, Predicted 1s: 889
Accuracy = 0.67245, TP = 713, TN = 256, FP = 176, FN = 296
Epoch: 4, Accuracy: 0.67245
Saving model
Predicted 0s: 734, Predicted 1s: 707
Accuracy = 0.59334, TP = 565, TN = 290, FP = 142, FN = 444
Epoch: 5, Accuracy: 0.59334
Predicted 0s: 560, Predicted 1s: 881
Accuracy = 0.65718, TP = 698, TN = 249, FP = 183, FN = 311
Epoch: 6, Accuracy: 0.65718
Predicted 0s: 739, Predicted 1s: 702
Accuracy = 0.53990, TP = 524, TN = 254, FP = 178, FN = 485
Epoch: 7, Accurac